In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd
import os
import cv2
import math
import xml.etree.ElementTree as X
import time

In [ ]:
#if any import error. Run previous cell and then restart the notebook kernel
from mmcv import collect_env
collect_env()

# Check MMRotate installation
import mmrotate
print(mmrotate.__version__)

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

In [ ]:
from mmrotate.datasets.builder import ROTATED_DATASETS
from mmrotate.datasets.dota import DOTADataset

@ROTATED_DATASETS.register_module()
class TinyDataset(DOTADataset):
    """SAR ship dataset for detection."""
    CLASSES = ('RI', 'RM', 'RR', 'RL', 'LI', 'LM', 'LR', 'LL', 'RT', 'LT',)

In [ ]:
# Install MMRotate from the source.
!git clone https://github.com/open-mmlab/mmrotate.git
%cd mmrotate

In [ ]:
# We use mim to download the pre-trained checkpoints for inference and finetuning.
!mim download mmrotate --config roi_trans_swin_tiny_fpn_1x_dota_le90 --dest .

In [ ]:
import mmcv
from mmcv.runner import load_checkpoint

from mmdet.apis import inference_detector, init_detector, show_result_pyplot
from mmrotate.models import build_detector

# Choose to use a config and initialize the detector
config = 'roi_trans_swin_tiny_fpn_1x_dota_le90.py'
# Setup a checkpoint file to load
checkpoint = 'roi_trans_swin_tiny_fpn_1x_dota_le90-ddeee9ae.pth'

# Set the device to be used for evaluation
device='cuda:0'

# Load the config
config = mmcv.Config.fromfile(config)
# Set pretrained to be None since we do not need pretrained model here
config.model.pretrained = None

# Initialize the detector
model = build_detector(config.model)

# Load checkpoint
checkpoint = load_checkpoint(model, checkpoint, map_location=device)

# Set the classes of models for inference
model.CLASSES = checkpoint['meta']['CLASSES']

# We need to set the model's cfg for inference
model.cfg = config

# Convert the model to GPU
model.to(device)
# Convert the model into evaluation mode
model.eval()

In [ ]:
from mmcv import Config
cfg = Config.fromfile('./configs/roi_trans/roi_trans_swin_tiny_fpn_1x_dota_le90.py')

In [ ]:
import mmcv
from mmcv.runner import load_checkpoint

from mmdet.apis import inference_detector, show_result_pyplot
from mmrotate.models import build_detector
from mmdet.apis import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'TinyDataset'
cfg.data_root = 'ssdd_tiny/'

cfg.data.test.type = 'TinyDataset'
cfg.data.test.data_root = 'ssdd_tiny/'
cfg.data.test.ann_file = 'val'
cfg.data.test.img_prefix = 'images'

cfg.data.train.type = 'TinyDataset'
cfg.data.train.data_root = 'ssdd_tiny/'
cfg.data.train.ann_file = 'train'
cfg.data.train.img_prefix = 'images'

cfg.data.val.type = 'TinyDataset'
cfg.data.val.data_root = 'ssdd_tiny/'
cfg.data.val.ann_file = 'val'
cfg.data.val.img_prefix = 'images'

#print(f'Config:\n{cfg.pretty_text}')
print(cfg.model.roi_head.bbox_head[1])
# modify num classes of the model in box head
cfg.model.roi_head.bbox_head[0].num_classes = 10
cfg.model.roi_head.bbox_head[1].num_classes = 10
# We can still use the pre-trained Mask RCNN model though we do not need to

# use the mask branch
#cfg.load_from = 'roi_trans_swin_tiny_fpn_1x_dota_le90-ddeee9ae.pth'
cfg.load_from = '/home/murshed/phd/git_research/ftsegment/rotated_object/transformer/mmrotate-main/demo/mmrotate/tutorial_exps/epoch_6.pth'



# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

cfg.optimizer.lr = 0.0001
cfg.lr_config.warmup = None
cfg.runner.max_epochs = 3
cfg.log_config.interval = 10

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 3
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 3

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device='cuda'

# We can also use tensorboard to log the training process
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='TensorboardLoggerHook')]

# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


In [ ]:
import os.path as osp

from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector


# Build dataset
datasets = [build_dataset(cfg.data.train)]

#print(cfg.data.train)

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))

In [ ]:
config_file = '../roi_trans_swin_tiny_fpn_1x_dota_le90_trained.py'
checkpoint_file = '../trained_model/transSEG.pth'
model = init_detector(config_file, checkpoint_file, device='cuda:0')

In [ ]:
img = mmcv.imread('../../crfseg/test_image/Image_00_1_1.bmp')
#img = mmcv.imread('/home/murshed/phd/git_research/ftsegment/datasets/fingerprint/raw_and_rotated_images/9_WT_150127-093901_15.jpeg')
model.cfg = cfg
result = inference_detector(model, img)
show_result_pyplot(model, img, result, score_thr=0.6)